### ** 포털공사관리번호 필요 ptlCmno (설계VE 목록 조회)

### 1. 기본 수집정보 불러오기

In [1]:
### 기본라이브러리 정의
from common import commonFunc as cf
import pandas as pd
import pickle

metadata = pd.read_excel("../input/datalake_meta22.xlsx")

### 기본정보 설정 및 파라미터 설정 ###
SITENAME = "건설사업정보시스템"
DATANAME= "설계VE 계약정보"
# APIKEY = "1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF"
### APIKEY 불러오기 ###
with open("../input/calsapikey.pickle","rb") as fr:
    APIKEY = pickle.load(fr)

targetData = metadata.loc[metadata.자료명==DATANAME]
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]
### 기본정보 설정 및 파라미터 설정 ###

### 2. 입력 파라미터 정보 불러오기

In [2]:
### 기본정보 설정 및 파라미터 설정 ###
PREDATANAME= "설계VE 목록 조회"
preData = metadata.loc[metadata.자료명==PREDATANAME]

preDt = preData.저장폴더.values[0]
preDtResult = pd.read_csv(preDt, encoding="ms949")
newParam = preDtResult.ptlcmno.drop_duplicates().tolist()

In [6]:
# 페이지번호 설정
PAGEYN=1
if REQPARAM.count("pageyn") == 0:
    PAGEYN = 0
else:
    PAGEYN = 1
### 기본정보 설정 및 파라미터 설정 ###

### [추가] 3. 파라미터정보 불러오기

In [7]:
orgParam = []

try:
    orgParam = cf.loadparam(SITENAME,DATANAME,SERVICENAME)
    outData = cf.loaddata(SITENAME,DATANAME,SERVICENAME)
## 파일이 없는경우
except Exception as e:
    orgParam = []
    print(e)

ptlcmnoList = list( set(newParam) - set(orgParam) )
# 모드: 0=종료 1=append 2=새로생성
mode = 2

# 업데이트 할 내용이 없으면 종료
if ptlcmnoList == []:
    mode = 0
    print("{} 정보 quit모드 {} ".format(DATANAME, mode))
    quit()
# orgParam == []
elif orgParam == []:
    mode = 2
    print("{} 정보 new 모드 {} ".format(DATANAME, mode))
# 이외에는 append 모드
else:
    mode = 1
    print("{} 정보 append 모드 {} ".format(DATANAME, mode))

[Errno 2] No such file or directory: '../output\\건설사업정보시스템\\설계VE 계약정보\\selectIoPtVeContractCorpList.pickle'
설계VE 계약정보 정보 new 모드 2 


### 데이터 수집

In [ ]:
### 개별 코드 작업 영역 ###
### baseparam은 수동설정필요

ptlcmnoListLen = len(ptlcmnoList)
resultDfMerged = pd.DataFrame()
for i in range(0, ptlcmnoListLen):
    BASEPARAM={"serviceKey":APIKEY, "type":"json", "ptlCmno": ptlcmnoList[i]}
    resultDf = cf.scrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN)
    resultDfMerged = resultDfMerged.append(resultDf)

1 page scraping start
Empty DataFrame
Columns: []
Index: []
1 page is empty
dataframe설계VE 계약정보, param:{'serviceKey': '5E24C9F4-66EA-4ADB-87F6-70D8AC71DAEA', 'type': 'json', 'ptlCmno': 'G9901102017001'} rows: 0 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe설계VE 계약정보, param:{'serviceKey': '5E24C9F4-66EA-4ADB-87F6-70D8AC71DAEA', 'type': 'json', 'ptlCmno': '15003092010004'} rows: 9 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
1 page is empty
dataframe설계VE 계약정보, param:{'serviceKey': '5E24C9F4-66EA-4ADB-87F6-70D8AC71DAEA', 'type': 'json', 'ptlCmno': 'B3800082018009'} rows: 0 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
1 page is empty
dataframe설계VE 계약정보, param:{'serviceKey': '5E24C9F4-66EA-4ADB-87F6-70D8AC71DAEA', 'type': 'json', 'ptlCmno': '61100002016002'} rows: 0 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe설계VE 계약정보, param:{'serviceKey': '5E24C9F4-6

### [추가] 파라미터 정보 추가저장

In [13]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME, mode)

resultDfMerged.shape

설계VE 계약정보 save compled


(57, 9)

In [28]:
# 기존사용된 파라미터 정보에 추가 파라미터 append 후 저장
ptlcmnoList = orgParam + ptlcmnoList

# 최종 파라미터 저장
cf.saveparam(ptlcmnoList, SITENAME,DATANAME,SERVICENAME)